# StyleGAN2-ADA-PyTorch

## Preface
This StyleGAN2-ADA-PyTorch repository (including this Colab notebook) was forked from [Derrick Schultz](https://github.com/dvschultz/stylegan2-ada-pytorch), which was forked from [Nvidia's original repo](https://github.com/NVlabs/stylegan2-ada-pytorch). Additional comments and instructions added by Doug Rosman.

## Initialize Notebook

In [3]:
# Required
# Connect to GPU instance

!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 4090 (UUID: GPU-4cc36cd3-6aa8-06a2-ec0b-dee66b991388)


In [ ]:
# Required
# Install dependencies

!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install timm==0.4.12 ftfy==6.1.1 ninja==1.10.2 opensimplex
!pip install tqdm
!pip install imageio-ffmpeg
!pip install click
!pip install requests
!pip install scipy
!pip install psutil

## Find a pre-trained model

- Training models from scratch takes a lot of time, so it's best to transfer learn from a pretrained model.
- Download any of the models below, and then upload the .pkl file to the `pretrained` folder in the repo.
- The dimensions of your images must match the dimensions of the pretrained model (1024x1024, 512x512, 256x256)
- The pretrained model you start with will have a subtle influence on the way your images look.

**Lists of pretrained models to choose from**
- https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ (Nvidia's official models (ada version))
  - [FFHQ](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl) (1024) (Flickr Faces High Quality, realistic human faces)
  - [MetFaces](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl) (1024) (Faces from paintings in the Met Museum of Art Collection)
  - [AFHQ Wild](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqwild.pkl) (512) (Animal Faces High Quality, realistic wild animal faces)
  - [AFHQ Cat](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqcat.pkl) (512) (Animal Faces High Quality, realistic cat faces)
  - [AFHQ Dog](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqdog.pkl) (512) (Animal Faces High Quality, realistic dog faces)
  - [BreCaHAD](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/brecahad.pkl) (512) (Breast Cancer microscopic biopsy images)

- https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/ (Nvidia's official models)
  - [Cat](https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-cat-config-f.pkl) (256)
  - [Church](https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-church-config-f.pkl) (256)
  - [Horse](https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-horse-config-f.pkl) (256)

- https://github.com/justinpinkney/awesome-pretrained-stylegan2 (collection by Justin Pinkney. There are more there than the few listed below)
  - [FFHQ 512](https://mega.nz/#!eQdHkShY!8wyNKs343L7YUjwXlEg3cWjqK2g2EAIdYz5xbkPy3ng) (512) (Lower resolution version of the FFHQ model)
  - [Flowers](https://drive.google.com/uc?id=13onBTt6xVwKmYTRCFbONBTbSXFqeiVcK) (512)
  - [Anime Portraits](https://mega.nz/#!PeIi2ayb!xoRtjTXyXuvgDxSsSMn-cOh-Zux9493zqdxwVMaAzp4) (512)

## Upload your dataset

- Before uploading, put all your images in a folder and compress the folder to a .zip file.
- Upload your zip file to the __datasets__ folder

__Zipping up your folders__

__Windows Users:__
1. Right-click the folder and select **Send to** --> **Compressed (zipped) folder**

__Mac Users:__

Zipping files on a Mac with the native compression tool creates a file called ".DS_Store" inside the folder, which causes some code not to work. I recommend installing a free piece of software called __Keka__.

1. Download and install Keka ([click to download](https://d.keka.io/))
1. Open Keka
1. Make sure the box next to **'Exclude Mac resource forks'** is checked
1. Drag and drop your dataset folder directly onto the open Keka app
1. It may ask you where to save your file. Select your desintation and compress your folder
1. Upload the .zip file to the **datasets** folder

![good zip with Keka](https://raw.githubusercontent.com/dougrosman/stylegan2-ada-pytorch/main/images/keka.png)



## Preparing your images for training (batch cropping + resizing)

In order to train, all your images
- must be the exact same dimensions
- must be square, either 256x256, 512x512 or 1024x1024, depending on the pretrained model you're using
- must be RGB images.

The following script resizes, crops, and converts your images to RGB.

In [ ]:
# Set your variables

width = 256  # EDIT THIS to your desired dimension (256, 512 or 1024)
height = width
dataset = "fruitsdata" # EDIT THIS to the name of your data set file (without the .zip)

data_src = "./datasets/" + dataset + ".zip"
data_dest = "./datasets/" + dataset + "-" + str(width) + ".zip" 

In [ ]:
# Run this after setting your variables to process your images

!python dataset_tool-jpg.py --source=$data_src --dest=$data_dest --width=$width --height=$height --transform="center-crop"

## Training your model

### Setting your variables

In [1]:
# Edit the following variables to make sure you're training with the correct settings.

# Required
# The name of your data set file (without the .zip)
dataset = "midday-v4"

# The size of your images (256, 512 or 1024)
size = 1024 

# the .pkl file you're transfer learning from. This file must be in the "pretrained" folder
resume_from = "metfaces.pkl"

# (OPTIONAL)
# set how frequently you save checkpoints. Lower == more frequent, but takes up more storage space
snapshot_count = 8

# set to 'True' to double your data set size by duplicating and mirroring each image horizontally. If your dataset includes text, or images that must face a certain direction, keep this set to false.
mirror_x = False 


# (DON'T CHANGE THESE UNLESS YOU KNOW WHAT YOU'RE DOING)
gamma_value = 50.0
augs = 'bg'
config = '11gb-gpu'

####
dataset_path = "./datasets/" + dataset + "-" + str(size) + ".zip"
output_dir = "./results"
resume_from = "./pretrained/" + resume_from
aug_strength = 0
train_count = 0 

### Setting your variables for resuming training (optional)


In [5]:
# Set these variables after running the initial "setting your variables" cell.

# From the file explorer, right click on your pickle file and select "copy path", then paste that below
resume_from = 'results/00015-midday-v1-1024-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000472.pkl'

# This number should match the number at the end of your pickle file (e.g. if you're resuming from network-snapshot-000200.pkl, you would set train_count to 200.
train_count = 472  

# You can find the aug_strength in the "log.txt" file of the training that produced the pickle file you're be resuming from. Scroll down to the bottom of log.txt. Select the aug strength that corresponds with the kimg number of your train_count above.
aug_strength = 2.545 

### Start Training


This is it, the moment we've been waiting for! Once you start your training, remember to keep this tab open and your computer on. Make sure you computer doesn't fall asleep.

In [2]:
# After setting your variables above, run this cell to start your training.
!python train.py --gpus=1 --cfg=$config --metrics=None --outdir=$output_dir --data=$dataset_path --snap=$snapshot_count --resume=$resume_from --augpipe=$augs --initstrength=$aug_strength --gamma=$gamma_value --mirror=$mirror_x --mirrory=False --nkimg=$train_count

^C


# Part 4: Generating Images and Videos

For an in-depth explanation of the latent space, interpolations, and the different ways to generate latent space walks, check out this video: [StyleGAN2 In-Depth Week 3 (latent spaces, linear interpolations and noise loops](https://www.youtube.com/watch?v=jKJCv9VGqLQ&t=1998s) by Artificial Images (Derrick Schultz). The link takes you to the part of the video where he begins to go through these. (33:18) 

## 4.0 Key Terms

- `network` - The pickle file to generate from. Network is another term for "trained model."
- `seed` - Our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation. Can be a number between 0 to 4,294,967,295. 
- `truncation` - How much to "truncate" the latent space. A measure of how "creative" or "realistic" the generator will be. Best results between -1 and 1. Low truncation (close to 0) produces a lower variety of images, adhering to the "average" image of your trained model. Higher truncation (close to 1) produces more varied images. Beyond -1 or 1, the results start getting more wonky, as these images are outside the bounds of the latent space.
- `interpolation` - Interpolation is a general concept of moving between two things. A trained model is a 512-dimension space, and *interpolating through* this space takes us through a range of images. There are different interpolation methods, each of which creates different "routes" through the latent space.

## 4.1 Single random images

This generates individual images for each seed value. This can be useful for finding a good starting image for the interpolation loops.

**Options**

`--outdir`: Output directory (folder) where your images will be saved.

`--trunc`: How much truncation, a number between -1 to 1 will produce the best results, but technically can be infinite. 

`--seeds`: The seeds to generate images from. Can use a range of values (e.g. 400-500), or comma-separated seed values (e.g. 10, 20, 30).

`--network`: Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).


In [ ]:
!python generate.py \
--outdir=./generated/random-1000-1020-noncon \
--trunc=1 \
--seeds=1000-1020 \
--network=./pretrained/cat256.pkl

## 4.2 Truncation Traversal



Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

**Options**

`--outdir`: Output directory (folder) where your images will be saved.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--seeds`: Pass this only one seed. Pick a favorite from your generated images.

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)

`--network`: Again, this should be the path to your .pkl file.

In [ ]:
!python generate.py \
--process="truncation" \
--outdir=./generated/trunc-1002_1004__-2-2 \
--start=-2 \
--stop=2 \
--increment=0.005 \
--seeds=1002 \
--network=./pretrained/cat256.pkl \
--fps=60

## 4.3 Linear Interpolation (Lerp)


A straight line from one point (seed) in the latent space to another. The distance from one seed to another will vary, so some interpolations will be fast, others slow.

**Options**

`--outdir`: Output directory (folder) where your images will be saved.

`--space`: "z" or "w". Depending on your model, w-space may interpolate more smoothly (like with faces).

`--trunc`: truncation value

`--seeds`: list of seeds to interpolate between. Comma separated. Each set of seeds will be given the same number of frames to complete the interpolation. Set the last seed to the same number as the first to create a seamless looping video.

`--network`: path to your .pkl file

`--frames`: How many frames to produce per interpolation. (e.g. if you set your frames to **30** and your seeds to 10, 20, 30, 10, this will create a video with **90** frames (10 to 20 (30 frames), 20 to 30 (30 frames), 30 to 10 (30 frames))

`--fps`: frames per second of your video (24, 30, 48, 60, 120, 144)

In [ ]:
!python generate.py \
--outdir=./generated/lerp-10_20_30_40_10-w2 \
--space="w" \
--trunc=0.75 \
--process="interpolation" \
--seeds=10,20,30,40,10 \
--network=./pretrained/cat256.pkl \
--frames=120 \
--fps=60

## 4.4 Noise Loop


If you want to just make a random but fun interpolation of your model the noise loop is the way to go. It creates a random path through the z space to show you a diverse set of images.

**Options**

`--outdir`: Output directory (folder) where your images will be saved.

`--trunc`: truncation value

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--random_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

`--network`: path to your .pkl file

`--frames`: How many frames to produce

`--fps`: frames per second of your video (24, 30, 48, 60, 120, 144)

In [1]:
!python generate.py \
--outdir=./generated/middayv4_192-noiseloop-s100-d5-t1 \
--trunc=1 \
--process="interpolation" \
--interpolation="noiseloop" \
--diameter=5 \
--random_seed=100 \
--network=./results/00018-midday-v4-1024-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000192.pkl \
--frames=4500 \
--fps=25

Loading networks from "./results/00018-midday-v4-1024-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000192.pkl"...
Generating image for frame 0/4500 ...
Setting up PyTorch plugin "bias_act_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/4500 ...
Generating image for frame 2/4500 ...
Generating image for frame 3/4500 ...
Generating image for frame 4/4500 ...
Generating image for frame 5/4500 ...
Generating image for frame 6/4500 ...
Generating image for frame 7/4500 ...
Generating image for frame 8/4500 ...
Generating image for frame 9/4500 ...
Generating image for frame 10/4500 ...
Generating image for frame 11/4500 ...
Generating image for frame 12/4500 ...
Generating image for frame 13/4500 ...
Generating image for frame 14/4500 ...
Generating image for frame 15/4500 ...
Generating image for frame 16/4500 ...
Generating image for frame 17/4500 ...
Generating image for fra

d:\GitHub\stylegan2-ada-pytorch\generate.py:59: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  elif(len(seeds) is not 3):
d:\GitHub\stylegan2-ada-pytorch\torch_utils\ops\bias_act.py:50: UserWarning: Failed to build CUDA kernels for bias_act. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "d:\GitHub\stylegan2-ada-pytorch\torch_utils\ops\bias_act.py", line 48, in _init
    _plugin = custom_ops.get_plugin('bias_act_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "d:\GitHub\stylegan2-ada-pytorch\torch_utils\custom_ops.py", line 110, in get_plugin
    torch.utils.cpp_extension.load(name=module_name, verbose=verbose_build, sources=sources, **build_kwargs)
  File "d:\GitHub\stylegan2-ada-pytorch\.conda\lib\site-packages\torch\utils\cpp_extension.py", line 1080, in load
    return _jit_compile(
  File "d:\GitHub\stylegan2-ada-pytorch\.conda\lib\site-packages\torch\utils\cpp_extension.py", line 1293, in _j

In [2]:
!python generate.py \
--outdir=./generated/middayv4_192-noiseloop-s100-d6-t1 \
--trunc=1 \
--process="interpolation" \
--interpolation="noiseloop" \
--diameter=6 \
--random_seed=100 \
--network=./results/00018-midday-v4-1024-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000192.pkl \
--frames=4500 \
--fps=25

Loading networks from "./results/00018-midday-v4-1024-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000192.pkl"...
Generating image for frame 0/4500 ...
Setting up PyTorch plugin "bias_act_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/4500 ...
Generating image for frame 2/4500 ...
Generating image for frame 3/4500 ...
Generating image for frame 4/4500 ...
Generating image for frame 5/4500 ...
Generating image for frame 6/4500 ...
Generating image for frame 7/4500 ...
Generating image for frame 8/4500 ...
Generating image for frame 9/4500 ...
Generating image for frame 10/4500 ...
Generating image for frame 11/4500 ...
Generating image for frame 12/4500 ...
Generating image for frame 13/4500 ...
Generating image for frame 14/4500 ...
Generating image for frame 15/4500 ...
Generating image for frame 16/4500 ...
Generating image for frame 17/4500 ...
Generating image for fra

d:\GitHub\stylegan2-ada-pytorch\generate.py:59: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  elif(len(seeds) is not 3):
d:\GitHub\stylegan2-ada-pytorch\torch_utils\ops\bias_act.py:50: UserWarning: Failed to build CUDA kernels for bias_act. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "d:\GitHub\stylegan2-ada-pytorch\torch_utils\ops\bias_act.py", line 48, in _init
    _plugin = custom_ops.get_plugin('bias_act_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "d:\GitHub\stylegan2-ada-pytorch\torch_utils\custom_ops.py", line 110, in get_plugin
    torch.utils.cpp_extension.load(name=module_name, verbose=verbose_build, sources=sources, **build_kwargs)
  File "d:\GitHub\stylegan2-ada-pytorch\.conda\lib\site-packages\torch\utils\cpp_extension.py", line 1080, in load
    return _jit_compile(
  File "d:\GitHub\stylegan2-ada-pytorch\.conda\lib\site-packages\torch\utils\cpp_extension.py", line 1293, in _j

In [3]:
!python generate.py \
--outdir=./generated/middayv4_192-noiseloop-s100-d7-t1 \
--trunc=1 \
--process="interpolation" \
--interpolation="noiseloop" \
--diameter=7 \
--random_seed=100 \
--network=./results/00018-midday-v4-1024-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000192.pkl \
--frames=4500 \
--fps=25

Loading networks from "./results/00018-midday-v4-1024-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000192.pkl"...
Generating image for frame 0/4500 ...
Setting up PyTorch plugin "bias_act_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/4500 ...
Generating image for frame 2/4500 ...
Generating image for frame 3/4500 ...
Generating image for frame 4/4500 ...
Generating image for frame 5/4500 ...
Generating image for frame 6/4500 ...
Generating image for frame 7/4500 ...
Generating image for frame 8/4500 ...
Generating image for frame 9/4500 ...
Generating image for frame 10/4500 ...
Generating image for frame 11/4500 ...
Generating image for frame 12/4500 ...
Generating image for frame 13/4500 ...
Generating image for frame 14/4500 ...
Generating image for frame 15/4500 ...
Generating image for frame 16/4500 ...
Generating image for frame 17/4500 ...
Generating image for fra

d:\GitHub\stylegan2-ada-pytorch\generate.py:59: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  elif(len(seeds) is not 3):
d:\GitHub\stylegan2-ada-pytorch\torch_utils\ops\bias_act.py:50: UserWarning: Failed to build CUDA kernels for bias_act. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "d:\GitHub\stylegan2-ada-pytorch\torch_utils\ops\bias_act.py", line 48, in _init
    _plugin = custom_ops.get_plugin('bias_act_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "d:\GitHub\stylegan2-ada-pytorch\torch_utils\custom_ops.py", line 110, in get_plugin
    torch.utils.cpp_extension.load(name=module_name, verbose=verbose_build, sources=sources, **build_kwargs)
  File "d:\GitHub\stylegan2-ada-pytorch\.conda\lib\site-packages\torch\utils\cpp_extension.py", line 1080, in load
    return _jit_compile(
  File "d:\GitHub\stylegan2-ada-pytorch\.conda\lib\site-packages\torch\utils\cpp_extension.py", line 1293, in _j

In [4]:
!python generate.py \
--outdir=./generated/middayv4_192-noiseloop-s200-d5-t1 \
--trunc=1 \
--process="interpolation" \
--interpolation="noiseloop" \
--diameter=5 \
--random_seed=200 \
--network=./results/00018-midday-v4-1024-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000192.pkl \
--frames=4500 \
--fps=25

^C


Loading networks from "./results/00018-midday-v4-1024-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000192.pkl"...
Generating image for frame 0/4500 ...
Setting up PyTorch plugin "bias_act_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for frame 1/4500 ...
Generating image for frame 2/4500 ...
Generating image for frame 3/4500 ...
Generating image for frame 4/4500 ...
Generating image for frame 5/4500 ...
Generating image for frame 6/4500 ...
Generating image for frame 7/4500 ...
Generating image for frame 8/4500 ...
Generating image for frame 9/4500 ...
Generating image for frame 10/4500 ...
Generating image for frame 11/4500 ...
Generating image for frame 12/4500 ...
Generating image for frame 13/4500 ...
Generating image for frame 14/4500 ...
Generating image for frame 15/4500 ...
Generating image for frame 16/4500 ...
Generating image for frame 17/4500 ...
Generating image for fra

d:\GitHub\stylegan2-ada-pytorch\generate.py:59: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  elif(len(seeds) is not 3):
d:\GitHub\stylegan2-ada-pytorch\torch_utils\ops\bias_act.py:50: UserWarning: Failed to build CUDA kernels for bias_act. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "d:\GitHub\stylegan2-ada-pytorch\torch_utils\ops\bias_act.py", line 48, in _init
    _plugin = custom_ops.get_plugin('bias_act_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "d:\GitHub\stylegan2-ada-pytorch\torch_utils\custom_ops.py", line 110, in get_plugin
    torch.utils.cpp_extension.load(name=module_name, verbose=verbose_build, sources=sources, **build_kwargs)
  File "d:\GitHub\stylegan2-ada-pytorch\.conda\lib\site-packages\torch\utils\cpp_extension.py", line 1080, in load
    return _jit_compile(
  File "d:\GitHub\stylegan2-ada-pytorch\.conda\lib\site-packages\torch\utils\cpp_extension.py", line 1293, in _j

In [ ]:
!python generate.py \
--outdir=./generated/middayv4_192-noiseloop-s200-d6-t1 \
--trunc=1 \
--process="interpolation" \
--interpolation="noiseloop" \
--diameter=6 \
--random_seed=200 \
--network=./results/00018-midday-v4-1024-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000192.pkl \
--frames=4500 \
--fps=25

In [ ]:
!python generate.py \
--outdir=./generated/middayv4_192-noiseloop-s200-d7-t1 \
--trunc=1 \
--process="interpolation" \
--interpolation="noiseloop" \
--diameter=7 \
--random_seed=200 \
--network=./results/00018-midday-v4-1024-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000192.pkl \
--frames=4500 \
--fps=25

## 4.5 Circular Loop

The noise loop is, well, noisy. This circular loop will feel much more even, while still providing a random loop.

I recommend using a higher `--diameter` value than you do with noise loops. Something between `50.0` and `500.0` alongside `--frames` can help control speed and diversity.

**Options**

`--outdir`: Output directory (folder) where your images will be saved.

`--trunc`: truncation value

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--random_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

`--network`: path to your .pkl file

`--frames`: How many frames to produce

`--fps`: frames per second of your video (24, 30, 48, 60, 120, 144)

In [ ]:
!python generate.py \
--outdir=./generated/circularloop_1003_d1000 \
--trunc=0.75 \
--process="interpolation" \
--interpolation="circularloop" \
--diameter=1000 \
--random_seed=1003 \
--network=./pretrained/cat256.pkl \
--frames=120 \
--fps=60

# Part 5: Projection


Project images into the latent space. The network looks at your image and tries to reverse engineer its latent vector in order to re-create your image from what the latent space understands about images.

**Steps:**
1. Find an image you want to project
2. Resize it so that it is the same dimensions as your trained model
3. Upload the image to your Google Drive in the stylegan2-ada-pytorch repository (in the generated/project/input folder)

**Options**

`--outdir`: Output directory (folder) where your images will be saved.

`--target`: Filepath to your input image

`--num-steps`: how many iterations the projector should run for. Lower will mean fewer steps and less likelihood of a good projection. Higher will take longer but will likely produce better images.

`--network`: Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).


In [ ]:
!python projector.py \
--outdir=./generated/projection/lea6 \
--target=./generated/projection/input/lea256.jpg \
--num-steps=30 \
--seed=20 \
--network=./pretrained/cat256.pkl \
--video-target-concat=True # set to True for side-by-side of target image + output video

# Part 6: Face Alignment

*Work in progress, eventually this script will accept a .zip folder.*

Align faces using the same process used to align faces for the FFHQ data set. Run this script if you plan to train a data set on faces, or project a face into a latent space with aligned faces like FFHQ, metfaces, or AFHQ (Wild, Dog or Cat) Although this should maybe go in the Data Set Preparation section, I'm including it here at the bottom since it's a more of a niche process that won't be run that frequently.

**Steps**
(again, this is a work in progress, so the process is a little less streamlined)




In [ ]:
# 1. change directory into the align_faces folder
%cd util/align_faces/

In [ ]:
# 2. "clean" your images by making sure the alignment script can process them
!python clean_images_downscale.py /path/to/dataset.zip 2000

In [ ]:
# 3. unzip your images to a folder called raw_images
!unzip output.zip -d raw_images

In [ ]:
# 4. align the faces, saving them to a folder called aligned
!python align_faces.py raw_faces aligned

In [ ]:
#5. (optional) zip up the aligned faces so they can be used for training
!zip -r aligned.zip aligned

In [ ]:
# change directory back into the main folder so that all the other scripts work
%cd /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch

# Convert Legacy Model



If you have an older version of a model (Tensorflow based StyleGAN, or Runway downloaded .pkl file) you’ll need to convert to the newest version. If you’ve trained in this notebook you do **not** need to use this cell.

`--source`: path to model that you want to convert

`--dest`: path and file name to convert to.

In [ ]:
!python legacy.py --source=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/cat256.pkl --dest=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/cat256-converted.pkl

## Combine NPZ files together

In [ ]:
!python combine_npz.py --outdir=/content/npz --npzs='/content/pb-proj1-3-clip.npz,/content/pb-proj1-clip-nocenter.npz,/content/pb-proj2-clip-nocenter.npz,/content/projected_w (2).npz'

# Random snippets

In [ ]:
!zip -r vid1.zip /content/out/video1-w-0.5

In [ ]:
!ffmpeg -y -r 144 -i ./generated/ffhq-144fps/frames/frame%04d.png -vcodec libx265 -crf 25 -preset faster -pix_fmt yuv420p ./generated/ffhq-144fps/lerp144.mp4

#### 4.4 Spherical Interpolation (Slerp) (Not Working)

This gets a little heady, but technically linear interpolations are not the best in high-dimensional GANs. [This github link](https://github.com/soumith/dcgan.torch/issues/14) is one of the more popular explanations and discussions.

That said, the differences between lerp and slerp aren't all that significant.

Note: Slerp in w space currently isn’t supported.

**Options**

`--outdir`: Output directory (folder) where your images will be saved.

`--trunc`: truncation value

`--seeds`: list of seeds to interpolate between. Comma separated. Each set of seeds will be given the same number of frames to complete the interpolation. Set the last seed to the same number as the first to create a seamless looping video.

`--network`: path to your .pkl file

`--frames`: How many frames to produce per interpolation. (e.g. if you set your frames to **30** and your seeds to 10, 20, 30, 10, this will create a video with **90** frames (10 to 20 (30 frames), 20 to 30 (30 frames), 30 to 10 (30 frames))

`--fps`: frames per second of your video (24, 30, 48, 60, 120, 144)

In [ ]:
!python generate.py \
--outdir=./generated/slerp-10_20_30_40_10-z \
--interpolation="slerp" \
--process="interpolation" \
--trunc=0.75 \
--seeds=10,20,30,40,10 \
--network=./pretrained/cat256.pkl \
--frames=120 \
--fps=60